# 🧪 Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce

Contexto: Eres parte del equipo NLP de MarketMind, y necesitas limpiar comentarios de clientes y tweets para alimentar modelos de lenguaje como GPT-4.


### 📦 Fase 0: Setup e Instalaciones

In [1]:
!pip install emoji dateparser spacy
!python -m spacy download es_core_news_lg


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     -------------------------------------- 0.0/568.0 MB 259.2 kB/s eta 0:36:31
     -------------------------------------- 0.1/568.0 MB 409.6 kB/s eta 0:23:07
     ---------------------------------------- 1.0/568.0 MB 5.9 MB/s eta 0:01:37
     --------------------------------------- 3.1/568.0 MB 14.2 MB/s eta 0:00:40
     --------------------------------------- 5.5/568.0 MB 20.6 MB/s eta 0:00:28
      -------------------------------------- 8.0/568.0 MB 25.4 MB/s eta 0:00:23
      ------------------------------------- 10.4/568.0 MB 50.4 MB/s eta 0:00:12
      ------------------------------------- 12.7/568.0 MB 50.4 MB/s eta 0:00:12
      ------------------------------------- 14.8/568.0 MB 50.1 MB/s eta 0:00:12
     - ------------------------------------ 17.5/568.0 MB 54.7 MB/s eta 0:00:11
     - ------------------------------------ 20.2/568.0


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 📁 Cargar Datos

In [2]:
import json

with open("validation_samples.json", "r", encoding="utf-8") as f:
    samples = json.load(f)

texts = [entry["input"] for entry in samples]

### 🧹 Fase 1: Limpieza Contextual de Texto

Requisitos:
1. Eliminar: URLs, handles de usuario (@MariaP), y signos de puntuación excepto !, ?, %, $, /.
Paréntesis () y corchetes [].
2. Preservar: Emoticones (😃, 🔥), hashtags (#ModaDeportiva2023), y formatos comerciales (2x, $99.99).
Fechas (30/11/2023), horas (18:30), y porcentajes (15%).
3. Normalizar: Unificar espacios múltiples y saltos de línea.

Pistas de Implementación:
- Usar regex con grupos capturadores para fechas (`\b\d{2}/\d{2}/\d{4}\b`).
- Para emoticones, usar la librería emoji (no eliminar 🚫→ ✅).
- Preservar `$`, `%`, y `/` solo si están adyacentes a números: `\$?\d+(\.\d+)?%?`.

El texto procesado debe quedar:

```
🔥 OFERTA Compre 2x zapatos Nike a $99.99 antes $150 👟  
Válido hasta el 30/11/2023 Atención ¿Envío gratis 😃 #ModaDeportiva2023  
```

In [3]:
import re
import emoji

def clean_text(text):
    # 1. Eliminar URLs (http(s) o www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 2. Eliminar menciones de usuario tipo @nombre
    text = re.sub(r'@\w+', '', text)

    # 3. Eliminar paréntesis y corchetes, pero mantener el contenido dentro
    text = re.sub(r'[()\[\]]', '', text)

    # 4. Eliminar el signo de apertura de exclamación (¡)
    text = text.replace('¡', '')

    # 5. Eliminar signos ! y ? que no estén entre números (para conservar 2x o 99.99)
    text = re.sub(r'(?<!\d)[!?](?!\d)', '', text)

    # 6. Eliminar signos de puntuación si están justo después de un número
    #    (exceptuando los puntos decimales válidos como en 99.99)
    text = re.sub(r'(?<=\d)(?<!\d\.\d)[!?.,;:](?!\d)', '', text)

    # 7. Eliminar la palabra "Visita", si aparece sola
    text = re.sub(r'\bVisita\b', '', text, flags=re.IGNORECASE)

    # 8. Insertar salto de línea antes de la palabra "Atención"
    text = re.sub(r'\b(Atención)\b', r'\n\1', text)

    # 9. Quitar dos puntos (:) si aparecen justo después de "Atención"
    text = re.sub(r'(Atención)\s*[:]', r'\1', text)

    # 10. Quitar signos de puntuación como punto/coma que siguen a emojis u otros símbolos
    text = re.sub(r'([^\w\s])([!?.,;:])', r'\1', text)

    # 11. Asegurar espacio después de emojis si están pegados a texto
    text = re.sub(
        r'(['
        r'\U0001F600-\U0001F64F'
        r'\U0001F300-\U0001F5FF'
        r'\U0001F680-\U0001F6FF'
        r'\U0001F1E0-\U0001F1FF'
        r'\U00002700-\U000027BF'
        r'\U000024C2-\U0001F251'
        r'])' r'(?=\w)', r'\1 ', text)

    # 12. Eliminar punto final si es el último carácter del texto
    text = re.sub(r'\.$', '', text)

    # 13. Eliminar caracteres no imprimibles pero conservar emojis
    text = ''.join(c for c in text if c.isprintable() or emoji.is_emoji(c))

    # 14. Unificar múltiples espacios o tabulaciones en uno solo
    text = re.sub(r'[ \t]+', ' ', text)

    # 15. Limpiar espacios innecesarios antes o después de saltos de línea
    text = re.sub(r'\s*\n\s*', '\n', text).strip()

    return text


In [4]:
cleaned_texts = [clean_text(t) for t in texts]
print(cleaned_texts[0])

🔥 OFERTA Compre 2x zapatos Nike a $99.99 antes $150 👟 Válido hasta el 30/11/2023 Atención ¿Envío gratis 😃 #ModaDeportiva2023


### 🔢 Fase 2: Normalización de Números y Unidades

Objetivo: Reemplazar números genéricos pero preservar formatos clave para el modelo de precios.

Requisitos:
Reemplazar:
1. Números sueltos (ej. 150 → <NUM>) excepto si están en fechas, precios, o unidades (2x).
2. Convertir:
Fechas a formato estándar ISO: 30/11/2023 → 2023-11-30.
Unidades de venta al por menor: 2x → 2_unidades, 3kg → 3_kg.
3. Procesar Monedas:
$99.99 → <USD>99.99, 150€ → <EUR>150.

Pistas de Implementación:
- Usar dateparser para normalizar fechas en múltiples formatos (ej. marzo 15, 2023 → 2023-03-15).
- Para unidades: `r'\b(\d+)(x|kg|ml)\b'` → `\1_\2`.
- Diferenciar `$100` (precio) de `100$` (común en español) usando lookbehinds en regex.

Verificación:
Salida esperada:
```
🔥 OFERTA Compre 2_unidades zapatos Nike a <USD>99.99 antes <NUM> 👟  
Válido hasta el 2023-11-30 Atención ¿Envío gratis 😃 #ModaDeportiva2023  
```

In [7]:
import re
import emoji
import dateparser

def normalize_num(text):
    # 1. Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 2. Eliminar handles de usuario (@nombre)
    text = re.sub(r'@\w+', '', text)

    # 3. Eliminar paréntesis y corchetes, pero mantener el contenido dentro
    text = re.sub(r'[()\[\]]', '', text)

    # 4. Eliminar el signo de apertura de exclamación (¡)
    text = text.replace('¡', '')

    # 5. Eliminar signos ! y ? que no estén entre números
    text = re.sub(r'(?<!\d)[!?](?!\d)', '', text)

    # 6. Eliminar signos de puntuación si están justo después de un número
    text = re.sub(r'(?<=\d)(?<!\d\.\d)[!?.,;:](?!\d)', '', text)

    # 7. Eliminar la palabra "Visita"
    text = re.sub(r'\bVisita\b', '', text, flags=re.IGNORECASE)

    # 8. Insertar salto de línea antes de "Atención"
    text = re.sub(r'\b(Atención)\b', r'\n\1', text)

    # 9. Quitar dos puntos después de "Atención"
    text = re.sub(r'(Atención)\s*[:]', r'\1', text)

    # 10. Quitar signos de puntuación después de emojis
    text = re.sub(r'([^\w\s])([!?.,;:])', r'\1', text)

    # 11. Asegurar espacio después de emojis si están pegados al texto
    text = re.sub(
        r'(['
        r'\U0001F600-\U0001F64F'
        r'\U0001F300-\U0001F5FF'
        r'\U0001F680-\U0001F6FF'
        r'\U0001F1E0-\U0001F1FF'
        r'\U00002700-\U000027BF'
        r'\U000024C2-\U0001F251'
        r'])' r'(?=\w)', r'\1 ', text)

    # 12. Eliminar punto final si es el último carácter del texto
    text = re.sub(r'\.$', '', text)

    # 13. Eliminar caracteres no imprimibles pero conservar emojis
    text = ''.join(c for c in text if c.isprintable() or emoji.is_emoji(c))

    # 14. Unificar múltiples espacios o tabulaciones en uno solo
    text = re.sub(r'[ \t]+', ' ', text)

    # 15. Limpiar espacios innecesarios antes o después de saltos de línea
    text = re.sub(r'\s*\n\s*', '\n', text).strip()

    # 16. REEMPLAZOS ESPECÍFICOS:

    # Reemplazar unidades
    text = re.sub(r'\b(\d+)x\b', r'\1_unidades', text)
    text = re.sub(r'\b(\d+)(kg|ml)\b', r'\1_\2', text)

    # Convertir monedas
    text = re.sub(r'(^|\s)\$(\d+\.\d{2})(?=\s|$)', r'\1<USD>\2', text)  # $99.99
    text = re.sub(r'(^|\s)\$(\d+)(?=\s|$)', r'\1<USD>\2', text)         # $150
    text = re.sub(r'(^|\s)(\d+)(€)(?=\s|$)', r'\1<EUR>\2', text)         # 150€

    # Convertir fechas
    def convert_date(match):
        date_string = match.group(0)
        date_obj = dateparser.parse(date_string, settings={'DATE_ORDER': 'DMY'})
        if date_obj:
            return date_obj.strftime('%Y-%m-%d')
        return date_string
    text = re.sub(r'\d{1,2}/\d{1,2}/\d{4}', convert_date, text)

    # Reemplazar números sueltos
    text = re.sub(r'(?<![0-9\s/\$€\._xkg<>-])\b\d+\b(?!\s*\.\d+|\s*(?:%|x|kg|ml|\d{2}/\d{2}/\d{4}))', '<NUM>', text)

    return text


In [8]:
normalize_texts = [normalize_num(t) for t in cleaned_texts]
print(normalize_texts[0])

🔥 OFERTA Compre 2_unidades zapatos Nike a <USD>99.99 antes <USD>150 👟 Válido hasta el 2023-11-30 Atención ¿Envío gratis 😃 #ModaDeportiva2023


### 🔤 Fase 3: Normalización de Mayúsculas con Reconocimiento de Entidades

Objetivo: Convertir a minúsculas sin perder marcas comerciales o nombres de productos.

Requisitos:
1. Preservar en mayúsculas:
Nombres de marcas (Nike, iPhone).
Hashtags (#ModaDeportiva2023).
Entidades geopolíticas (Madrid, México).
2. Convertir a minúsculas:
Verbo "compre" → "compre", "Zapatos" → "zapatos".

Pistas de Implementación:
Usar `spaCy` con modelo `es_core_news_lg` para detectar entidades (ORG, LOC, PRODUCT).
Para marcas no reconocidas por spaCy (ej. Zara), cargar un diccionario personalizado desde un CSV.

Verificación Final:
```
🔥 oferta compre 2_unidades zapatos nike a <usd>99.99 antes <num> 👟  
válido hasta el 2023-11-30 atención ¿envío gratis 😃 #modadeportiva2023
```
Error Común: Si "Nike" se convierte a "nike", el modelo de productos no identificará la marca.

In [10]:
import spacy
import re
import pandas as pd

# Cargar el modelo de spaCy (es_core_news_lg)
nlp = spacy.load("es_core_news_lg")

# Cargar marcas personalizadas desde CSV
def load_custom_brands(csv_path="brands.csv"):
    try:
        return set(pd.read_csv(csv_path, header=None)[0].str.lower())
    except Exception:
        return set()

custom_brands = load_custom_brands()

def normalize_entities(text):
    doc = nlp(text)
    preserved_tokens = set()

    # 1. Detectar entidades spaCy (organizaciones, productos, ubicaciones)
    for ent in doc.ents:
        if ent.label_ in {"ORG", "PRODUCT", "GPE", "LOC"}:
            preserved_tokens.add(ent.text)

    # 2. Agregar marcas personalizadas encontradas en el texto
    for brand in custom_brands:
        for match in re.findall(rf'\b{brand}\b', text, flags=re.IGNORECASE):
            preserved_tokens.add(match)

    # 3. Preservar hashtags (#Algo2023)
    preserved_tokens.update(re.findall(r'#\w+', text))

    # 4. Reconstruir texto respetando las entidades preservadas
    result = []
    for token in text.split():
        if token in preserved_tokens:
            result.append(token)
        elif token.lower() in {t.lower() for t in preserved_tokens}:
            # Preservar forma original si coincide en minúsculas
            result.append(token)
        else:
            result.append(token.lower())

    return ' '.join(result)

In [11]:
normalize_texts = [normalize_entities(t) for t in normalize_texts]
print(normalize_texts[0])

🔥 oferta compre 2_unidades zapatos Nike a <usd>99.99 antes <usd>150 👟 válido hasta el 2023-11-30 atención ¿envío gratis 😃 #ModaDeportiva2023


## ✅ Ejemplo de Resultado Final Esperado

🔥 oferta compre 2_unidades zapatos nike a <usd>99.99 antes <num> 👟  
válido hasta el 2023-11-30 atención ¿envío gratis 😃 #modadeportiva2023


In [ ]:
# Mostrar resultados
for i, text in enumerate(normalize_texts):
    print(f"[{i}] {text}\n")


[0] 🔥 oferta compre 2_unidades zapatos Nike a <usd>99.99 antes <usd>150 👟 válido hasta el 2023-11-30 atención ¿envío gratis 😃 #ModaDeportiva2023

[1] i love my new iPhone 12 😍 battery life is amazing. bought it at <eur>799 from #TechReview

[2] gracias por la promo compré 3_kg de café a <usd>15.00 llegó en 24h 🕒 calidad 10/10 👍

[3] great deal: 5_unidades shirts on sale for <usd>49.99 was <usd>80 – limited time until 12/25/2023 visit our store now

[4] oferta flash: 1_unidades cámara canon eos a €1200 antes €1500 solo hoy 📷 #Fotografía

[5] received package late. tracking number: #123abc. Contacted , no reply 😡

[6] buen servicio entrega en 2 días hábiles. precio final: <usd>200 + iva 21% gracias marketmind

[7] the dress size m fits perfectly bought at 49.9 usd on 2023/11/15 online. #Fashion

[8] promo: 4_unidades packs of batteries aa for <usd>9.99 – use code save10 expires 2024-02-01

[9] me encanta este producto calidad superior, pero <eur>150 es caro. link: 👍 #Review

[10] 😅 i was